# QandA Demo

## Libraries etc

In [1]:
import os
import pandas as pd
from pathlib import Path

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore

from qanda import QandA
from scores import calculate_bertscore_df

## The QandA Object

A `QandA` object is a RAG (Retrieval-Augmented Generation) question-answer chain encapsulated in a simple Python object. It exposes a single primary method, `ask()`, for ease of use.

## Initialize a QandA Object
To create an instance of the `QandA` class, you need to provide the following parameters:
- `file_path`: The path to your document (`.jsonl` format).
- `gen_model`: The generative LLM you wish to use (e.g., "gemma3", "llama3.2").
- `embed_model`: The embedding model for the vector store.
- `vdb`: The vector store class to use (e.g., `InMemoryVectorStore`).
- `top_k`: The number of relevant document chunks to retrieve for context.
- `prompt`: The prompt template for the language model.

In [2]:
FILE_PATH = Path("jsondata/Rodier-Finding.jsonl")
GEN_MODEL = "gemma3"
EMBED_MODEL = "mxbai-embed-large"
VDB = InMemoryVectorStore
TOP_K = 3

PROMPT = ChatPromptTemplate.from_template(
    """Context information is below.\n
    ---------------------\n
    {context}\n
    ---------------------\n
    Given the context information and not prior knowledge, answer the query.\n
    Query: {input}\n
    Answer:\n"""
)

Now we can instantiate the `QandA` object.

In [3]:
qanda = QandA(gen_model=GEN_MODEL,
              embed_model=EMBED_MODEL, 
              vdb=VDB,
              file_path=FILE_PATH,
              top_k=TOP_K,
              prompt=PROMPT)

Initializing, please wait...
Loading jsondata\Rodier-Finding.jsonl
Question Answer chain ready.


The object is now ready. We can inspect its documentation using the `help()` function to understand its capabilities.

In [4]:
help(qanda)

Help on QandA in module qanda object:

class QandA(builtins.object)
 |  QandA(gen_model, embed_model, vdb, file_path, top_k, prompt)
 |
 |  A class for performing question-answering tasks using a language model and a vector database.
 |
 |  Attributes:
 |      gen_model (str): The name of the language model to be used for generating answers.
 |      embed_model (str): The name of the embedding model to be used for generating embeddings.
 |      vdb (str): The name of the vector database to be used for storing and retrieving documents.
 |      file_path (str): The path to the file containing the documents to be used for the question-answering task.
 |      top_k (int): The number of top-k most relevant documents to be retrieved for each question.
 |      prompt (str): The prompt to be used for the question-answering chain.
 |
 |  Methods:
 |      ask(question, verbose=False):
 |          Invokes the question-answering chain to generate an answer to the given question.
 |          Args:


## How to Use the QandA Object

### 1. Basic Usage: Getting an Answer

The simplest way to use the object is to call the `ask()` method with a question. This will return the generated answer as a string.

In [5]:
QUESTIONS = [
    "Who is the coroner?", 
    "Who is the deceased?", 
    "What was the cause of death?"
]

for question in QUESTIONS:
    answer = qanda.ask(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: Who is the coroner?
Answer: Sarah Helen Linton, Deputy State Coroner.

Question: Who is the deceased?
Answer: Frank Edward Rodier is the deceased.

Question: What was the cause of death?
Answer: The cause of death remains unascertained. The report states: “Accordingly, his cause of death must remain unascertained.”



### 2. Verbose Usage: Getting the Answer and Sources

To verify the answer and see which parts of the document were used as context, you can set `verbose=True`. This returns a tuple containing the answer and a list of source documents.

In [9]:
for question in QUESTIONS:
    answer, sources = qanda.ask(question, verbose=True)
    
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("Sources:")
    for src in sources:
        
        text_preview = src['text'].replace('\n', ' ')[:200] + '...'
        print(f"  Source {src['source']}:")
        print(f"    - Text: {text_preview}")
        print(f"    - Page: {src['page']}")
        print(f"    - Document: {src['document']}")
    print("\n" + "-" * 50)

Question: Who is the coroner?
Answer: Sarah Helen Linton, Deputy State Coroner.
Sources:
  Source 1:
    - Text: Counsel Appearing: Senior Constable C Robertson assisted the Coroner . Case(s) referred to in decision(s): Nil...
    - Page: 1
    - Document: data/Rodier-Finding.pdf
  Source 2:
    - Text: [2024] WACOR 35 Coroners Act 1996 (Section 26(1))...
    - Page: 2
    - Document: data/Rodier-Finding.pdf
  Source 3:
    - Text: [2024] WACOR 35 JURISDICTION CORONER'S COURT OF WESTERN AUSTRALIA ACT CORONERS ACT 1996 CORONER SARAH HELEN LINTON, DEPUTY STATE CORONER HEARD 14 AUGUST 2024 DELIVERED 14 AUGUST 2024 FILE NO/S CORC 32...
    - Page: 1
    - Document: data/Rodier-Finding.pdf

--------------------------------------------------
Question: Who is the deceased?
Answer: Frank Edward Rodier is the deceased.
Sources:
  Source 1:
    - Text: IS DEATH ESTABLISHED? 17. As is clear from the above; I am satisfied beyond reasonable doubt that Frank Rodier is deceased and that he died on 25

### 3. Programmatic Evaluation

For more systematic testing, we can compare the model's answers against a set of known correct answers. Here, we use the `bert_score` library to numerically evaluate the semantic similarity between the generated answer and the ground truth.

The process involves:
1. Defining a list of questions and their corresponding correct answers.
2. Generating answers from the LLM for each question.
3. Creating a pandas DataFrame to hold the questions, correct answers, and LLM answers.
4. Using the `calculate_bertscore_df` function from the `scores` module to compute precision, recall, and F1 scores.

In [8]:
#  Defining questions and ground truth answers
CORRECT_ANSWERS = [
    "Sarah Helen Linton",
    "Frank Edward Rodier",
    "unascertained"
]


print("Generating LLM answers for evaluation...")
llm_answers = [qanda.ask(q) for q in QUESTIONS]

data = {
    'FILENAME': [FILE_PATH.stem] * len(QUESTIONS),
    'MODEL': [GEN_MODEL] * len(QUESTIONS),
    'QUESTION': QUESTIONS,
    'CORRECT_ANSWER': CORRECT_ANSWERS,
    'LLM_ANSWER': llm_answers
}
df = pd.DataFrame(data)
scores_df = calculate_bertscore_df(df)

# results
print("Evaluation Results:")
display(scores_df)



Generating LLM answers for evaluation...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.67 seconds, 1.79 sentences/sec
Evaluation Results:


,FILENAME,MODEL,QUESTION,CORRECT_ANSWER,LLM_ANSWER,BERT_PRECISION,BERT_RECALL,BERT_F1
0,Rodier-Finding,gemma3,Who is the coroner?,Sarah Helen Linton,"Sarah Helen Linton, Deputy State Coroner.",0.888055,0.959326,0.922316
1,Rodier-Finding,gemma3,Who is the deceased?,Frank Edward Rodier,Frank Edward Rodier is the deceased.,0.913599,0.961740,0.937052
2,Rodier-Finding,gemma3,What was the cause of death?,unascertained,The cause of death remains unascertained. The ...,0.808263,0.863161,0.834811
